#### New to Plotly?
Plotly's Python library is free and open source! [Get started](https://plot.ly/python/getting-started/) by downloading the client and [reading the primer](https://plot.ly/python/getting-started/).
<br>You can set up Plotly to work in [online](https://plot.ly/python/getting-started/#initialization-for-online-plotting) or [offline](https://plot.ly/python/getting-started/#initialization-for-offline-plotting) mode, or in [jupyter notebooks](https://plot.ly/python/getting-started/#start-plotting-online).
<br>We also have a quick-reference [cheatsheet](https://images.plot.ly/plotly-documentation/images/python_cheat_sheet.pdf) (new!) to help you get started!

# Cars dataset exploration with plotly.py version 3.0

## Load cars dataset
The dataset for this example, as well as the ipynb files are available in [this github repo](https://github.com/jonmmease/plotly_ipywidget_notebooks).

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go

cars_df = pd.read_csv('data/cars/cars.csv',
                      usecols=['City mpg',
                               'Fuel Type',
                               'Horsepower',
                               'Model Year',
                               'Torque', 'Hybrid', 'ID'])
cars_df.sample(5)

,City mpg,Fuel Type,Horsepower,Hybrid,ID,Model Year,Torque
4579,27,Gasoline,155,False,2012 Mazda Mazda3 4-Door i Touring,2012 Mazda Mazda3 4-Door,148
3532,12,Gasoline,403,False,2010 GMC Sierra 1500 Crew Cab Short Box SLT 4W...,2010 GMC Sierra 1500,417
940,17,Gasoline,265,False,2010 Ford Edge SEL AWD,2010 Ford Edge,250
2770,19,Gasoline,210,False,2010 Saab 9-3 2.0T SportCombi AT,2010 Saab 9-3 SportCombi,221
2128,21,Gasoline,171,False,2010 Mazda Tribute i Sport AT,2010 Mazda Tribute,171


In [2]:
cars_df.shape

(5076, 7)

## Load images of cars

In [3]:
import os

image_data = {}
for img_filename in os.listdir('data/cars/images'):
    model_year = img_filename.split('.')[0]
    with open(f"data/cars/images/{img_filename}", "rb") as f:
        b = f.read()
        image_data[model_year] = b

In [ ]:
from ipywidgets import Image
Image(value=image_data['2012_Chevrolet_Camaro_Coupe'])

## Construct plotly.py Figure Widget
Torqe vs. MPG Scatter Trace

In [5]:
import plotly.graph_objs as go

In [6]:
fig = go.FigureWidget(
    data=[
        dict(
            type='scattergl',
            x=cars_df['Torque'],
            y=cars_df['City mpg'],
            mode='markers',
        )
    ],
)

### Display Figure
Before online or offline `iplot`. Still supported, but not needed with `FigureWidget`

In [7]:
fig

FigureWidget({
    'data': [{'mode': 'markers',
              'type': 'scattergl',
              'uid': '24406…

### Label Figure
Use property assignment syntax to:

Set `fig.layout.title` to `'Torque and Fuel Efficience'`

In [31]:
fig.layout.title = 'Torque and Fuel Efficience'

Check default font size

In [32]:
fig.layout.titlefont.size

17

Increase the title font size

In [33]:
fig.layout.titlefont.size = 22

Set `fig.layout.titlefont.family` to `'Rockwell'`

In [34]:
fig.layout.titlefont.family = 'Rockwell'

### Create New View for Figure
If working in JupyterLab, right-click on blue bar to the left of the figure and select "Create New View for Output". Drag view to the right half of the screen.

### Label Axes

Set the `fig.layout.xaxis.title` property to `'Torque (foot-pounds)'`

In [35]:
fig.layout.xaxis.title = 'Torque (foot-pounds)'

Set the `fig.layout.yaxis.title` property to `'City MPG'`

In [36]:
fig.layout.yaxis.title = 'City MPG'

### Notice Quantization
Zoom in and notice that the dataset is quantized

### Apply Jitter

In [ ]:
scatter = fig.data[0]
scatter

In [38]:
N = len(cars_df)
scatter.x = scatter.x + np.random.rand(N) * 10
scatter.y = scatter.y + np.random.rand(N) * 1

Zoom level did not reset! Plot is updated in-place. Not recreated each time a property changes

### Address Overplotting

Lower marker opacity

In [39]:
scatter.marker.opacity = 0.2

Decrease marker size

In [40]:
scatter.marker.size = 4

### Aside on validation

What if I though opacity ranged from 0 to 255?

In [41]:
# scatter.marker.opacity = 50

What if I forgot the name of an enumeration value?

In [42]:
# fig.layout.hovermode = 'nearest' # Set to 'closest'
fig.layout.hovermode = 'closest'

What if I don't know how to spell 'fuchsia'?

In [43]:
scatter.marker.color = 'fuchsia' # Set to 'fuchsia'

Restore default marker color

In [44]:
scatter.marker.color = None

### Add density contour

Add smoothed density contour trace (`histogram2dcontour`) based on `scatter.x` and `y=scatter.y` values.

In [22]:
contour = fig.add_histogram2dcontour(
    x=scatter.x, y=scatter.y)

Set contour colorscale

In [23]:
contour.colorscale = 'Hot'

Reverse the colorscale

In [24]:
contour.reversescale = True

Disable tooltips for contour

In [25]:
contour.hoverinfo = 'skip'

Tweak marker size and opacity

In [26]:
scatter.marker.opacity = .1
scatter.marker.size = 3

### Create marker configuration widget

Define function that inputs `opacity` and `size` and updates the figure.

In [27]:
def set_opacity(opacity, size):
    scatter.marker.opacity = opacity
    scatter.marker.size = size

Use `ipywidgets.interactive` to generate control panel for function.

In [28]:
from ipywidgets import interactive
opacity_slider = interactive(set_opacity,
                             opacity=(0.0, 1.0, 0.01),
                             size=(1, 10, 0.25))
opacity_slider

interactive(children=(FloatSlider(value=0.5, description='opacity', max=1.0, step=0.01), FloatSlider(value=5.0…

Adjust the width of the slider widgets

In [29]:
opacity_slider.children[0].layout.width = '400px'
opacity_slider.children[1].layout.width = '400px'

Try zooming and then adjusting the marker params

### Looking at outliers

#### Tooltips
Use `'ID'` column as tooltip for scatter

In [30]:
scatter.text = cars_df['ID']
scatter.hoverinfo = 'text'

#### All properties

Create an HTML widget to display the hover properties

In [31]:
from ipywidgets import HTML
details = HTML()
details

HTML(value='')

Register callback function to be executed on hover events. It will update the HTML widget using the pandas `to_html` method.

In [32]:
def hover_fn(trace, points, state):
    ind = points.point_inds[0]
    details.value = cars_df.iloc[ind].to_frame().to_html()

scatter.on_hover(hover_fn)

#### Vehicle image

Create an `ipywidgets.Image` widget to display images

In [33]:
from ipywidgets import Image, Layout
image_widget = Image(
    value=image_data['2012_Chevrolet_Camaro_Coupe'],
    layout=Layout(height='252px', width='400px')
)
image_widget

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x08\x06\x0…

Update hover function to update the image widget along with the HTML widget

In [34]:
def hover_fn(trace, points, state):

    ind = points.point_inds[0]
    
    # Update details HTML widget
    details.value = cars_df.iloc[ind].to_frame().to_html()
    
    # Update image widget
    model_year = cars_df['Model Year'][ind].replace(' ', '_')
    image_widget.value = image_data[model_year]

scatter.on_hover(hover_fn)

## Bringing it all together

Create simple dashboard using `HBox` and `VBox` containers

In [ ]:
from ipywidgets import HBox, VBox
VBox([fig,
      opacity_slider,
      HBox([image_widget, details])])

<img src='https://raw.githubusercontent.com/michaelbabyn/plot_data/master/cars_exploration.gif'>

#### Reference

See [these Jupyter notebooks](https://github.com/jonmmease/plotly_ipywidget_notebooks) for even more FigureWidget examples.

In [3]:
help(go.FigureWidget)

Help on class FigureWidget in module plotly.graph_objs._figurewidget:

class FigureWidget(plotly.basewidget.BaseFigureWidget)
 |  Method resolution order:
 |      FigureWidget
 |      plotly.basewidget.BaseFigureWidget
 |      plotly.basedatatypes.BaseFigure
 |      ipywidgets.widgets.domwidget.DOMWidget
 |      ipywidgets.widgets.widget.Widget
 |      ipywidgets.widgets.widget.LoggingHasTraits
 |      traitlets.traitlets.HasTraits
 |      traitlets.traitlets.HasDescriptors
 |      __builtin__.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, data=None, layout=None, frames=None)
 |      Create a new FigureWidget instance
 |      
 |      Parameters
 |      ----------
 |      data
 |          The 'data' property is a tuple of trace instances
 |          that may be specified as:
 |            - A list or tuple of trace instances
 |              (e.g. [Scatter(...), Bar(...)])
 |            - A list or tuple of dicts of string/value properties where:
 |              - The 'ty

In [2]:
from IPython.display import display, HTML

display(HTML('<link href="//fonts.googleapis.com/css?family=Open+Sans:600,400,300,200|Inconsolata|Ubuntu+Mono:400,700" rel="stylesheet" type="text/css" />'))
display(HTML('<link rel="stylesheet" type="text/css" href="http://help.plot.ly/documentation/all_static/css/ipython-notebook-custom.css">'))

! pip install git+https://github.com/plotly/publisher.git --upgrade
    
import publisher
publisher.publish(
    'cars-exploration.ipynb', 'python/cars-exploration/', 'Car Exploration with go.FigureWidget, Case Study',
    'Use Plotly FigureWidget with hover callbacks and slider widgets',
    title = 'Car Exploration with Hover Events',
    name = 'Car Exploration with Hover Events',
    has_thumbnail='true', thumbnail='thumbnail/figurewidget-cars.gif', 
    language='python',
    display_as='chart_events', order=26,
    ipynb= '~notebook_demo/242')

  Cloning https://github.com/plotly/publisher.git to /tmp/pip-req-build-5e_v1khw
  Running setup.py bdist_wheel for publisher ... done
  Stored in directory: /tmp/pip-ephem-wheel-cache-vaj2gwxn/wheels/99/3e/a0/fbd22ba24cca72bdbaba53dbc23c1768755fb17b3af0f33966
Successfully built publisher
  Found existing installation: publisher 0.13
    Uninstalling publisher-0.13:
      Successfully uninstalled publisher-0.13
You are using pip version 10.0.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
